# Face count


In [1]:
import cv2

# Load the pre-trained Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize video capture (0 is the default camera)
cap = cv2.VideoCapture(0)

# Variable to store the number of faces detected
face_count = 0

# Function to detect faces and update the face count
def detect_faces(frame):
    global face_count
    # Convert frame to grayscale for better detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Update the face count with the number of faces detected
    face_count = len(faces)
    
    # Draw rectangles around the faces detected
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Return the modified frame
    return frame

# Main loop for processing the video stream
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # Detect faces and update the frame
    frame = detect_faces(frame)
    
    # Display the current face count on the frame
    cv2.putText(frame, f'Face Count: {face_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Show the frame with the face count
    cv2.imshow('Face Counter', frame)
    
    # Break the loop if the user presses 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()


# Face Lock

In [2]:
import cv2

# Load the pre-trained Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize video capture (0 is the default camera)
cap = cv2.VideoCapture(0)

# Parameters
line_position = 250  # Y-coordinate of the counting line
offset = 30  # Offset to create tolerance for crossing detection
people_count = 0  # To store the count of people entering

# Store the previous Y-coordinate of faces to track direction
previous_faces = []

# Function to detect faces and count people entering
def detect_and_count_faces(frame):
    global people_count, previous_faces
    
    # Convert frame to grayscale for better face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    current_faces = []
    
    # Draw the counting line
    cv2.line(frame, (0, line_position), (frame.shape[1], line_position), (0, 255, 0), 2)
    
    # Process each face detected
    for (x, y, w, h) in faces:
        # Draw rectangle around each face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        
        # Calculate the center of the face
        cx = int(x + w / 2)
        cy = int(y + h / 2)
        
        # Draw the center point
        cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
        
        # Store the Y-coordinate of the center of the face for current frame
        current_faces.append(cy)
        
        # Check if face crosses the line from below to above (entering)
        for prev_y in previous_faces:
            if prev_y < line_position and cy >= line_position - offset:
                people_count += 1
                break
    
    # Update the list of previous face Y-coordinates for the next frame
    previous_faces = current_faces
    
    return frame

# Main loop to process video
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # Resize frame to make it easier to process
    frame = cv2.resize(frame, (640, 480))
    
    # Detect faces and count people
    frame = detect_and_count_faces(frame)
    
    # Display the count of people on the frame
    cv2.putText(frame, f'Face Detected Locked', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    # Show the video frame with detection
    cv2.imshow('Face Detection', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


# Face Recog


In [1]:
import cv2
import face_recognition
import numpy as np
import os

In [3]:

# Initialize variables to store registered face encodings and their names
known_face_encodings = []
known_face_names = []

# Directory to store registered face images
REGISTRATION_DIR = 'registered_faces'
if not os.path.exists(REGISTRATION_DIR):
    os.makedirs(REGISTRATION_DIR)

# Function to register a face with a name
def register_face(name):
    # Start the webcam
    video_capture = cv2.VideoCapture(0)
    print("Press 'q' to capture your face for registration.")
    
    while True:
        ret, frame = video_capture.read()
        cv2.imshow('Video - Register Face', frame)
        
        # Wait for the user to press 'q' to capture
        if cv2.waitKey(1) & 0xFF == ord('q'):
            # Save the captured image
            image_path = os.path.join(REGISTRATION_DIR, f"{name}.jpg")
            cv2.imwrite(image_path, frame)
            break
    
    video_capture.release()
    cv2.destroyAllWindows()
    
    # Load the captured image and encode the face
    image = face_recognition.load_image_file(image_path)
    face_encoding = face_recognition.face_encodings(image)[0]
    
    # Store the face encoding and name
    known_face_encodings.append(face_encoding)
    known_face_names.append(name)

# Function to recognize faces in a live video feed
def recognize_faces():
    video_capture = cv2.VideoCapture(0)
    
    while True:
        ret, frame = video_capture.read()
        rgb_frame = frame[:, :, ::-1]  # Convert BGR to RGB for face_recognition
        
        # Find all faces and their encodings in the current frame
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
        
        # Loop over each face found in the frame to check if it matches a known face
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            # Compare face encoding to known faces
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"
            
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
            
            # Draw a box around the face and label with name
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
            cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 1.0, (255, 255, 255), 1)
        
        # Display the video feed
        cv2.imshow('Video - Face Recognition', frame)
        
        # Exit the feed when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    video_capture.release()
    cv2.destroyAllWindows()

# Main function to interact with the user
def main():
    while True:
        print("\nChoose an option:")
        print("1: Register a new face")
        print("2: Start video feed and recognize faces")
        print("q: Quit")
        
        choice = input("Enter your choice: ")
        
        if choice == '1':
            name = input("Enter the name of the person to register: ")
            register_face(name)
        elif choice == '2':
            if len(known_face_encodings) == 0:
                print("No faces registered yet. Please register a face first.")
            else:
                recognize_faces()
        elif choice == 'q':
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == '__main__':
    main()



Choose an option:
1: Register a new face
2: Start video feed and recognize faces
q: Quit
No faces registered yet. Please register a face first.

Choose an option:
1: Register a new face
2: Start video feed and recognize faces
q: Quit
No faces registered yet. Please register a face first.

Choose an option:
1: Register a new face
2: Start video feed and recognize faces
q: Quit
Press 'q' to capture your face for registration.

Choose an option:
1: Register a new face
2: Start video feed and recognize faces
q: Quit

Choose an option:
1: Register a new face
2: Start video feed and recognize faces
q: Quit
